# Framework Overview

**Framework Control Flow**: **`conftest.py`** >> **`Base`** class >> test class
* Define setup & tear down in **`conftest.py`** for browser invocation and return an instance of **`webdriver`** to be used in **`Base`** class.
* The **`Base`** class will be use the **`setup`** fixture defined in **`conftest.py`** to access the **`webdriver`** instance.
* Also, define common utilites in **`Base`** class required for the test classed.
* The test classes will inherit the **`Base`** class to access the **`webdriver`** instance and common utilities.
* Add a pytest hook in **`conftest.py`** to handle command-line options for selecting the browser at run-time.
* Implements Page Object design pattern.
* Implements logging feature to tests
* Test execution HTML reporting
* Automatic screenshot capture on test failure
* Integrating selenium python framework to Jenkins CI tools with build parameterization

# **`conftest.py`**

```
# pytest hook: to handle command-line option

def pytest_addoption(parser):
    parser.addoption(
        "--browser", action="store", default="chrome"
    )

# fixture passing driver object to the test class

@pytest.fixture(scope="class")
def setup(request):                     # pytest provides request parameter for handling runtime properties

    browser = request.config.getoption("browser")
    if browser == 'firefox':
        service_obj = Service("../resources/geckodriver.exe")
        driver = webdriver.Firefox(service=service_obj)
    else:
        service_obj = Service("../resources/chromedriver.exe")
        driver = webdriver.Chrome(service=service_obj)
    driver.maximize_window()
    driver.get("https://rahulshettyacademy.com/angularpractice/")

    # setting driver variable/property in request parameter
    # now the method/class which will use this fixture will have access to this driver object
    request.cls.driver = driver

    yield
    time.sleep(2)
    driver.close()
```

# **`base.py`**

```
import pytest
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.wait import WebDriverWait


@pytest.mark.usefixtures("setup")
class Base:

    def verifyLinkPresence(self,text):
        wait = WebDriverWait(self.driver, 10)       # using driver instance received from the setup fixture

        wait.until(expected_conditions.presence_of_element_located((By.LINK_TEXT, text)))
```

# **`test.py`**

```
import pytest
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.wait import WebDriverWait
from utilities.BaseClass import BaseClass


class TestOne(Base):
    def test_e2e(self):

        # setting local driver with webdriver instance inherited from the Base class which is coming from conftest.py
        driver = self.driver

        driver.find_element(By.CSS_SELECTOR, "a[href*='shop']").click()
```

# Page Object pattern

## **`home.py`**

```
from selenium.webdriver.common.by import By


class Home:

    def __init__(self, webdriver):
        self.driver = webdriver

    shop = (By.CSS_SELECTOR, "a[href*='shop']")

    def shopItems(self):
        return self.driver.find_element(*Home.shop) # * - for deserializing them from tuple

```

## **`checkout.py`**

```
from selenium.webdriver.common.by import By


class Checkout:

    def __init__(self, webdriver):
        self.driver = webdriver

    products = (By.CSS_SELECTOR, ".card-title a")
    add_product_button = (By.CSS_SELECTOR, ".card-footer button")

    def getAllProducts(self):
        return self.driver.find_elements(*Checkout.products)

    def getAllProductsButton(self):
        return self.driver.find_elements(*Checkout.add_product_button)

```

## **`test.py`**

```
import pytest
from selenium.webdriver.common.by import By
from utilities.BaseClass import BaseClass
from pages.home import Home



class TestTwo(BaseClass):
    def test_form(self,getData):

        # setting local driver with webdriver instance inherited from the Base class which is coming from conftest.py
        driver = self.driver

        # Using HOME page object model class
        home = Home(driver)
        driver.refresh()
        driver.find_element(By.NAME, "email").send_keys(getData[2])
        driver.find_element(By.ID, "exampleInputPassword1").send_keys(getData[1])
        driver.find_element(By.ID, "exampleCheck1").click()


    # Parameterizing webdriver tests with multiple data sets

    @pytest.fixture(params=[("Kiranmoy","13579", "kiran@gmail.com"),("Paul","24680", "paul@gmail.com")])
    def getData(self, request):
        return request.param
```